In [30]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq groq
import os
from groq import Groq
import getpass
client = Groq()

In [31]:
EXPERT_CONFIGS = {
    "technical": {
        "system_prompt": "You are a Senior Software Engineer. Provide rigorous, code-focused, and precise solutions. Always check for syntax and logic errors.",
        "model": "llama-3.1-8b-instant"
    },
    "billing": {
        "system_prompt": "You are a Billing Specialist. You are empathetic, focused on financial accuracy, and strictly adhere to company refund and subscription policies.",
        "model": "llama-3.1-8b-instant"
    },
    "general": {
        "system_prompt": "You are a helpful General Support Assistant. Use a friendly tone and provide concise answers for non-technical and non-billing queries.",
        "model": "llama-3.1-8b-instant"
    },
    "tool_use": {
        "system_prompt": "You are a Tool Use expert. You are designed to use specialized tools to answer specific queries.",
        "model": "llama-3.1-8b-instant"
    }
}

In [34]:
import random

def route_prompt(user_input):
    routing_instructions = (
        "Classify this text into one of these categories: [technical, billing, general, tool_use]. "
        "For queries related to fetching external data or using specific tools, such as 'current price of Bitcoin', classify them as 'tool_use'. "
        "Return ONLY the word of the category, nothing else."
    )

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": routing_instructions},
            {"role": "user", "content": user_input}
        ],
        model="llama-3.1-8b-instant",
        temperature=0
    )

    category = response.choices[0].message.content.strip().lower()
    return category

def process_request(user_input):

    category = route_prompt(user_input)
    print(f"--- [Router Log]: Routing to '{category}' expert ---")

    if category == "tool_use":
        return get_current_bitcoin_price()

    if category not in EXPERT_CONFIGS:
        category = "general"

    expert = EXPERT_CONFIGS[category]

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": expert["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        model=expert["model"],
        temperature=0.7
    )

    return response.choices[0].message.content

def get_current_bitcoin_price():
    mock_price = round(random.uniform(50000, 70000), 2)
    return f"The current price of Bitcoin is ${mock_price:,.2f} USD."

print(get_current_bitcoin_price())

The current price of Bitcoin is $52,039.86 USD.


In [33]:
# Test Case 1: Technical
print("\nInput: 'My python script is throwing an IndexError on line 5.'")
print("Response:", process_request("My python script is throwing an IndexError on line 5."))

print("-" * 30)

# Test Case 2: Billing
print("\nInput: 'I was charged twice for my subscription this month.'")
print("Response:", process_request("I was charged twice for my subscription this month."))

print("-" * 30)

# Test Case 3: General
print("\nInput: 'What time do you close on Fridays?'")
print("Response:", process_request("What time do you close on Fridays?"))

print("-" * 30)

# Test Case 4: Tool Use
print("\nInput: 'What is the current price of Bitcoin?'")
print("Response:", process_request("What is the current price of Bitcoin?"))


Input: 'My python script is throwing an IndexError on line 5.'
--- [Router Log]: Routing to 'technical' expert ---
Response: To help you troubleshoot the issue, I'll need more information. Can you please provide the following:

1. The code that's throwing the `IndexError`.
2. The line of code where the error is occurring (which you mentioned is line 5).
3. The exact error message you're seeing.
4. Any relevant context or surrounding code that might be contributing to the issue.

Once I have this information, I can help you identify the cause of the `IndexError` and provide a solution.

Here's a general example of how you might diagnose the issue:

```python
# Your code here

try:
    # Code that throws the IndexError
    print(data[5])  # This is line 5
except IndexError as e:
    print(f"Error: {e}")
    print(f"Line: 5")
    print(f"Data: {data}")
```

This will help you identify the specific data and line of code causing the issue.
------------------------------

Input: 'I was char